In [ ]:
!pip install ultralytics

In [ ]:
%cd /opt/app-root/src/
!yolo train model=yolov8n.pt batch=2 epochs=100 data=flyingthings.yaml exist_ok=True

In [ ]:
from PIL import Image
import IPython.display as display

# Set the path to the JPG file
image_path = 'runs/detect/train/results.png'

# Open the image file
image = Image.open(image_path)

# Display the image
display.display(image)


In [ ]:
%cd /opt/app-root/src/
!yolo predict model=runs/detect/train/weights/best.pt source=datasets/training/valid/images

In [ ]:
import os
import minio
import urllib3

# Disable SSL certificate verification
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set up MinIO client with insecure HTTPS
minio_endpoint = os.environ.get('minio-endpoint')
access_key = os.environ.get('access_key')
secret_key = os.environ.get('secret_key')
minio_client = minio.Minio(
    endpoint=minio_endpoint,
    access_key=access_key,
    secret_key=secret_key,
    secure=True,
    http_client=urllib3.PoolManager(cert_reqs='CERT_NONE', assert_hostname=False)
)


# Set the bucket name and file path
bucket_name = 'flyingthings'
file_path = 'runs/detect/train/weights/best.pt'

# Set the object name (the name under which the file will be stored in the bucket)
object_name = 'flyingthings.pt'

# Upload the file
minio_client.fput_object(bucket_name, object_name, file_path)

print(f"The file '{object_name}' has been uploaded to the '{bucket_name}' bucket.")
